<a href="https://colab.research.google.com/github/Sari275/my-deep-learning-projects/blob/main/TensorFlow_Program_1_Implementation_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from sklearn.metrics import matthews_corrcoef, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
train_path = "/content/train.dat"
test_path = "/content/test.dat"
train_data = pd.read_csv(train_path, header=None, sep='\t')
test_data = pd.read_csv(test_path,header=None, sep='\t')

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.dat'

In [ ]:
peptide = train_data[1].values
y = 1*(train_data[0].values > 0)

In [ ]:
pep_train, pep_valid, y_train, y_valid = train_test_split(peptide, y, random_state=123, test_size=0.2, stratify=y)
#print(y_valid)

In [ ]:
vec = CountVectorizer(tokenizer=lambda x: list(x), ngram_range=(1,2))

X_train = vec.fit_transform(pep_train).toarray()
X_valid = vec.transform(pep_valid).toarray()

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)

In [ ]:
X_train.shape, X_valid.shape
#X_train.shape[1]

((1252, 427), (314, 427))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(X_train.shape[1]),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(1e-4)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(loss='bce', optimizer='adam')

In [ ]:
es = tf.keras.callbacks.EarlyStopping(patience=5)
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=40, callbacks=[es])

Epoch 1/40
40/40 [==============================] - 2s 8ms/step - loss: 0.2768 - val_loss: 0.1416
Epoch 2/40
40/40 [==============================] - 0s 4ms/step - loss: 0.0930 - val_loss: 0.0855
Epoch 3/40
40/40 [==============================] - 0s 4ms/step - loss: 0.0388 - val_loss: 0.0808
Epoch 4/40
40/40 [==============================] - 0s 4ms/step - loss: 0.0263 - val_loss: 0.0881
Epoch 5/40
40/40 [==============================] - 0s 5ms/step - loss: 0.0225 - val_loss: 0.0852
Epoch 6/40
40/40 [==============================] - 0s 7ms/step - loss: 0.0205 - val_loss: 0.0859
Epoch 7/40
40/40 [==============================] - 0s 6ms/step - loss: 0.0197 - val_loss: 0.0864
Epoch 8/40
40/40 [==============================] - 0s 6ms/step - loss: 0.0191 - val_loss: 0.0870


In [ ]:
y_prob = model.predict(X_valid, verbose=False)
y_pred = 1*(y_prob > 0.5)
print('MCC:',matthews_corrcoef(y_valid, y_pred))
print('Accuracy:',accuracy_score(y_valid, y_pred))

MCC: 0.9005468707320919
Accuracy: 0.9840764331210191


In [ ]:
X_test = vec.transform(test_data[0].values).toarray()
X_test = sc.transform(X_test)

In [ ]:
y_prob = model.predict(X_test)
y_test = 1*(y_prob.ravel() > 0.5)


13/13 [==============================] - 0s 2ms/step


In [ ]:
pd.Series(y_test).to_csv('test.txt', index=False, header=None)